In [63]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer

In [64]:
seed = 42
iteration = 3000
threshold = 0.055

In [65]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("submission.csv").drop(columns='id')

In [66]:
def label_encoding(df: pd.DataFrame, columns: list) -> pd.Series:
    new_df = df.copy()
    for column in columns:
        le = LabelEncoder()
        value = le.fit_transform(df[column])
        new_df.loc[df[column].isna(),[column]]='NaN'
        new_df[column] = value
        
    return new_df

In [67]:
def split_customer_country_in_dataframe(df, column_name='customer_country'):
    def split_customer_country(customer_country):
        parts = customer_country.split('/')
        if len(parts) >= 3:
            city = parts[1].strip() if parts[1].strip() else 'NaN'
            country = parts[2].strip()
            return city, country
        return 'NaN', 'NaN'

    df[['city', 'country']] = df[column_name].apply(lambda x: pd.Series(split_customer_country(x) if pd.notnull(x) else (None, None)))
    return df

In [68]:
def update_strategic_versions(df):
    if 'idit_strategic_ver' not in df.columns:
        df['idit_strategic_ver'] = 0
    
    def apply_changes(row):
        if row['business_unit'] == 'ID':
            row['id_strategic_ver'] = 1
            row['it_strategic_ver'] = 0
            row['idit_strategic_ver'] = 1
        elif row['business_unit'] == 'IT':
            row['id_strategic_ver'] = 0
            row['it_strategic_ver'] = 1
            row['idit_strategic_ver'] = 1
        return row
    df = df.apply(apply_changes, axis=1)
    return df

In [69]:
#df_train['ver_pro_cus'] = df_train['ver_pro'] * df_train['ver_cus']
#df_test['ver_pro_cus'] = df_test['ver_pro'] * df_test['ver_cus']

In [70]:
#df_train = split_customer_country_in_dataframe(df_train, 'customer_country').drop(columns=['customer_country', 'customer_country.1'])
#df_test = split_customer_country_in_dataframe(df_test, 'customer_country').drop(columns=['customer_country', 'customer_country.1'])
df_all = pd.concat([df_train, df_test]).reset_index(drop=True).drop(columns=['customer_country.1'])

In [71]:
outlier_dict = {'fl': 'unitedstates', 'nevada': 'unitedstates', '85wmainstsuitec,canton,ga30114,': 'unitedstates', '230highlandave,suite531somervillema2143,': 'unitedstates',
                'ironhorsecustomsllc4443genellawaynorthlasvegas,nv89031,': 'unitedstates', '5301stevenscreekblvdsantaclaraca95051': 'unitedstates', '7700westsunriseblvdplantationfl33322': 'unitedstates', 'mo64108.': 'unitedstates', '3440rockefellerctwaldorf,md20602': 'unitedstates',
                'ca91915-6002': 'unitedstates', '2877prospectrd,fortlauderdale,fl33309': 'unitedstates', '9landsdownestreetbostonma2215': 'unitedstates', '160gouldstste300,needhamheights,massachusetts02494needhamma2721': 'unitedstates', 'oneconstitutionroadbostonma2129': 'unitedstates',
                '36marginstpeabodyma1960': 'unitedstates', '400centrestreetnewtonma2458': 'unitedstates', '1755northbrownrd.suite200lawrenceville,ga30043': 'unitedstates', '6kimballlanelynnfieldma1940': 'unitedstates', '1275sistergroverdvanalstyne,tx75495': 'unitedstates',
                '3nassonavenue': 'unitedstates', '810nkingstondrpeoria,il61604-2145': 'unitedstates', 'mi48827': 'unitedstates', '45n200wwillardut84340': 'unitedstates', '1385nweberrd,romeoville,il60446,': 'unitedstates',
                'ma02780': 'unitedstates', '152bowdoinstreet': 'unitedstates', '9820huntersvillenc28078': 'unitedstates', '3000montourchurchroad': 'unitedstates', '1380enterprisedr': 'unitedstates',
                '6601carrollhighlandsrd': 'unitedstates', '275mishawumroad': 'unitedstates', '222maxinedr': 'unitedstates', '2900highway280suite250birminghamal35223': 'unitedstates', 'il60069': 'unitedstates',
                '100vestaviaparkwaybirminghamal35216': 'unitedstates', '1100itbprovout84602': 'unitedstates', 'us': 'unitedstates', '700patrooncreekblvdalbanyny12206': 'unitedstates', '252beechavenuemelrosema2176': 'unitedstates',
                'caymanislands': 'unitedkingdom', 'britishvirginislands': 'unitedkingdom', 'bermuda': 'unitedkingdom', 'anguilla': 'unitedkingdom',
                'hanoi': 'vietnam', 'turkey': 'türkiye', '1605ave.poncedeleón,suite400sanjuan,00909,puertorico': 'puertorico', 'riodejaneiro': 'brazil',
                'mumbai': 'india', 'gujarat': 'india', 'telangana': 'india', 'unitedstates': 'unitedstates', 'θέσηπέτσαβακαλοπούλουβιοπαπαλλήνης15351': 'greece', 'madrid': 'spain',
                'agost,alicante': 'spain', '9631libertyrdb,randallstown,md21133': 'unitedstates', 'benbrook,tx76126': 'unitedstates', 'sc29555': 'unitedstates', '6252egrantrdsuite150tucson,az85712': 'unitedstates',
                ',mo64802': 'unitedstates', 'in46601,ee.uu.': 'unitedstates', 'warren,oh44483.': 'unitedstates', '1600rosecransavebldg7ste101,manhattanbeach,ca90266,': 'unitedstates', 'or97128': 'unitedstates', 'dourados': 'brazil',
                'fozdeiguaçu-pravenidatancredoneves6731jardimitaipu': 'brazil', 'recife': 'brazil', 'sãopaulo,pinheiros.': 'brazil', 'cartagena': 'colombia', 'co80127': 'unitedstates', 'cuiabá': 'brazil',
                'colombiacartagena': 'colombia', 'ny': 'unitedstates', 'carrera11a94-46edificiochico3000piso3bogota': 'colombia', 'va22209': 'unitedstates', 'ohio': 'unitedstates', 'caceres': 'spain',
                'ga31405': 'unitedstates', 'il61615': 'unitedstates', 'squareat,2662gatewayrdsuite165,carlsbad,ca92009': 'unitedstates', 'mo63103': 'unitedstates', 'country': np.nan, 'sd57751': 'unitedstates',
                'lucknow': 'india', 'chennai': 'india', 'tx75098': 'unitedstates', 'mo64506': 'unitedstates', 'ga30039': 'unitedstates', 'nv89119': 'unitedstates', 'nm': 'unitedstates', 'nj': 'unitedstates', 'elche,alicante': 'spain',
                '48201': np.nan, 'rj': 'unitedstates', 'colombiac2:soloinformación': 'colombia', 'bari,italy': 'italy', 'stcloud,mn56303': 'unitedstates', '609medicalcenterdr.decatur,texas,76234': 'unitedstates', '275johnhancockrd.taunton,ma,02780': 'unitedstates', 'pune': 'india', 'firenze,italy': 'italy',
                '3centerplzsuite330boston,ma02108': 'unitedstates', '1919minnesotact,mississauga,onl5n': 'canada', '233southbeaudryavenuelosangelesca': 'unitedstates', 'tx': 'unitedstates', 'englewood,co': 'unitedstates', '5003ladyofthelakedrraleighnc': 'unitedstates', 'ne': 'unitedstates', 'aparecida': 'brazil',
                'colombia-cartagena': 'colombia', 'capãodacanoa': 'brazil', 'bucaramanga': 'colombia', 'sãopaulo': 'brazil', 'joãopessoa': 'brazil', 'centrodeproduçãoaudiovisual-sescsãopaulo': 'brazil', 'saopaulo': 'brazil', '2367n2650wfarrwestut84404': 'unitedstates',
                '4880stevenscreekblvdsanjoseca95129': 'unitedstates', '450riverchasepkwybirminghamal35186': 'unitedstates', '1397etowahdriveatlantaga30319': 'unitedstates', 'tn38120': 'unitedstates', '823gatewaycenterway,sandiego,ca92102': 'unitedstates', 'isleofman': 'unitedkingdom', 'odisha': 'india', 'belohorizonte': 'brazil',
                'barranquilla': 'colombia', '9716mcfarringdrfortworthtx76244': 'unitedstates', '1999sbascombavesuite1000campbellca95008': 'unitedstates', 'pobox112292carrolltontx75011': 'unitedstates', '30winterstreetboston,ma02108': 'unitedstates', '1036nnalderst': np.nan, '1hoagdr.': 'unitedstates',
                '545west111thstsuite7c': 'unitedstates', '21903ranierln': 'unitedstates', '200cabellouisvilleky40206': 'unitedstates', '9820northcrosscentercthuntersvillenc28078': 'unitedstates', '5555': np.nan, '1503lbjparkwaysuite700farmersbranchtx75234': 'unitedstates',
                '6111wplanopkwy#2100planotx75093': 'unitedstates', '100sabineriverdr.huttotx78634': 'unitedstates', '6300harryhinesblvdste.1400dallastx75235': 'unitedstates', '100firststsanfranciscoca94015': 'unitedstates', '899kiferroadsunnyvaleca94086': 'unitedstates', '829jacksonave': 'unitedstates', '3027westbayvillaave': 'unitedstates', '126diabloranchcourt': 'unitedstates',
                '1537rollinghillsdr.': 'unitedstates', '1715forestcovedrive,apt201': 'unitedstates', '6005commercedr.ste.300': 'unitedstates', '17215welbyway': 'unitedstates', '416panzanodrive': 'unitedstates', '15806longshipct': 'unitedstates', '5501headquartersdrplanotx75024': 'unitedstates', '825eastlakeavee': 'unitedstates',
                '602sabercreekdrive': 'unitedstates', 'vt05672': 'unitedstates', 'nd': 'unitedstates', 'br': 'unitedstates', 'kerela': 'india', 'uttarpradesh': 'india', 'anandvihardelhi': 'india', 'hyderabad': 'india',
                'bhilwara': 'india', '9110forestcrossingthewoodlandstx77381': 'unitedstates', 'arlington,ny12603': 'unitedstates', 'grancanariasplayadelingles': 'spain', '24082carmeldr': 'unitedstates', '41720thstnbirminghamal35203': 'unitedstates', '594howardstsanfranciscoca94105': 'unitedstates', '594howardstsanfranciscoca94106': 'unitedstates',
                '305johnstreet': 'unitedstates', 'cra.51#12sur-75,sanfernando,itagüi,medellín,guayabal,medellín,antioquia,colombia': 'colombia', 'il60191': 'unitedstates', 'zip98433': 'unitedstates', '2475washingtonblvdogdenut84401': 'unitedstates', '723svalleyway,palmer,ak99645': 'unitedstates', 'tx77024': 'unitedstates', '1skyviewdrfortworthtx76155': 'unitedstates',
                '1209derbyruncarrollton,tx75007carrolltontx75007': 'unitedstates', '9111cypresswatersblvddallastx75038': 'unitedstates', '2350airportfrwybedfordtx76022': 'unitedstates', '117bernalrdste70-422sanjosesanjoseca95119': 'unitedstates', '1808lithgowrdcelinatx75009': 'unitedstates', '1909forestknolldrhooveral35244': 'unitedstates', '106lakeviewdrhomewoodal35209': 'unitedstates', '65grovestreet,suite204watertown,ma02472': 'unitedstates',
                '13854lakesidecirsterlingheights,mi48313': 'unitedstates', 'indore': 'india', 'ks66217': 'unitedstates', 'nj07013': 'unitedstates', 'ny11358': 'unitedstates', 'gurgaon': 'india', '1112badgervinelanearlingtontx76005': 'unitedstates', '14700caribbeanway': 'unitedstates',
                '335leaguests,sulphursprings,tx75482': 'unitedstates', '300eastparkdrive': 'unitedstates', '6005thstreet': 'unitedstates', 'ma01851': 'unitedstates', '1001mainst': 'unitedstates', 'gurgaon': 'india', 'viadell\'informatica10-37036sanmartinobuonalbergo(veneto),italy': 'italy', '1100leeave,lafayette,la70501,': 'unitedstates',
                '7105northlandterracen,minneapolis,mn55428': 'unitedstates', 'ca95814': 'unitedstates', '6564headquartersdrplanotx75051': 'unitedstates', '1275sistergroverdvanalstyne,tx75495': 'unitedstates', 'ca': 'unitedstates', '101metlifeway,cary,nc,27513–met1': 'unitedstates', '230highlandave,suite531somervillema2143': 'unitedstates', 'netherlandsantilles': 'netherlands',
                'valencia': 'spain', '3100shoredrivevirginiabeach,va23451': 'unitedstates', 'nicolosi(ct),italy': 'italy', '450riverchasepkwybirminghamal35186': 'unitedstates', '7673hempstoncir': 'unitedstates', 'sc29555': 'unitedstates', '2047wsummerdaleave': 'unitedstates', 'mn55024': 'unitedstates', '2266palmerdr': 'unitedstates', '955powellavesw': 'unitedstates', 'a': np.nan,
               '603heritagedrivemountjuliet': 'unitedstates', '136sindustrialsalinemi48176': 'unitedstates', '463industrialparkrd,elysburg,pa17824,us': 'unitedstates', '3131briarparkdrsuite200houstontx77042': 'unitedstates', 'fl33013': 'unitedstates',
               'p.o.box291992,portorange,fl32129': 'unitedstates', 'busshed,6501redhookrd#201,nazareth,stthomas00802,u.s.virginislands': 'unitedstates', 'fl32703': 'unitedstates', 'fl33025': 'unitedstates', '1800congressave.,austin,tx78701': 'unitedstates',
               'w126n7449flintdrivemenomoneefallsva': 'unitedstates', 'manaus': 'brazil', 'sãopaulo,pinheiros': 'brazil', '8003rdave3rdfloor,newyork,ny10022': 'unitedstates', 'usvirginislands': 'unitedstates',
               '30cambriaave,pleasantville,nj08232': 'unitedstates', '9800s.monroestreetsandyut84070': 'unitedstates', '11330clayrdhoustontx77041': 'unitedstates', '9420westsamhoustonpkwynhoustontx77018': 'unitedstates', '2217houstondrivemelissatx75454': 'unitedstates',
               'fl33442.': 'unitedstates', 'fl33716': 'unitedstates', 'ca95618': 'unitedstates', '9420westsamhoustonpkwynhoustontx77018': 'unitedstates', 'newhampshire': 'unitedstates',
               '2266palmerdr.': 'unitedstates', '8454muirwoodtrlfortworthtx76137': 'unitedstates', '410baylorstaustintx78703': 'unitedstates', '9420westsamhoustonpkwynhoustontx77018': 'unitedstates', 'ca92078': 'unitedstates',
               '750floridacentralparkwaysuite#100longwood,fl32750': 'unitedstates', 'fl33404': 'unitedstates', '724wbusinessushighway60,dexter,mo63841,': 'unitedstates', '400centrestnewtonma2458': 'unitedstates', '77massachusettsavecambridgema2139': 'unitedstates',
               '210route4eastfl4 ': 'unitedstates', 'fl33404': 'unitedstates', 'viae.deamicis,23.90044carini(pa)': 'italy', 'ironhorsecustomsllc4443genellawaynorthlasvegas,nv89031': 'unitedstates', 'herndon,va20170': 'unitedstates',
               '3801ewillowst,longbeach,ca90815,ee.uu.': 'unitedstates', '12718kittentrail,hudson,fl34669': 'unitedstates', 'jacksonvilleflorida': 'unitedstates', '4278sbuffalostorchardpark,ny14127': 'unitedstates', '1156warmitageavesuiteb,chicago,il60614,us.': 'unitedstates',
               'turksandcaicosislands': 'unitedkingdom', 'fl33772': 'unitedstates', 
                }



for idx, value in enumerate(df_all['customer_country']):
    if type(value) != float:
        new_value = value.split('/')[-1].replace(' ', '').lower()
        if new_value == '':
            df_all.loc[idx, ['customer_country']]=np.nan
        elif '.com' in new_value:
            df_all.loc[idx, ['customer_country']] = np.nan
        elif '.net' in new_value:
            df_all.loc[idx, ['customer_country']] = np.nan
        elif 'usa' in new_value:
            df_all.loc[idx, ['customer_country']] = 'unitedstates'
        elif 'unitedstates' in new_value:
            df_all.loc[idx, ['customer_country']] = 'unitedstates'
        else:
            df_all.loc[idx, ['customer_country']] = new_value
        for key in outlier_dict:
            if key == new_value:
                df_all.loc[idx, ['customer_country']]= outlier_dict[key]    
    
#ban_list = country[country==1].index
#df_all.loc[(df_all['customer_country'].isin(ban_list)), ['customer_country']]=np.nan

country = df_all['customer_country'].value_counts()
train_country = df_all[:len(df_train)]['customer_country'].value_counts()

In [72]:
columns_to_transform = ['com_reg_ver_win_rate', 
                        'historical_existing_cnt', 
                        'lead_desc_length'
                       ]

df_train[columns_to_transform] = df_train[columns_to_transform].fillna(df_train[columns_to_transform].median())

# pre_transformer = QuantileTransformer(
#     output_distribution='normal', 
#     random_state=seed
# )

pre_transformer = PowerTransformer(method='yeo-johnson')

df_train[columns_to_transform] = pre_transformer.fit_transform(df_train[columns_to_transform])
# df_train[columns_to_transform].head()

In [73]:
inquiry_conditions = {
    ('other_', 'Other', 'Others'): 'other',
    ('ETC.', 'Etc.', 'etc.'): 'etc',
    ('Sales inquiry',): 'Sales Inquiry',
    ('Quotation or purchase consultation', 'quotation_or_purchase_consultation', 'Quotation or Purchase Consultation', 'Purchase or Quotation', 'Request for quotation or purchase'): 'Quotation or Purchase consultation',
    ('usage_or_technical_consultation', 'usage or technical consultation', 'Usage or Technical Consultation', 'technical_consultation'): 'Usage or technical consultation'
}

customer_type_conditions = {
    ('Specifier/ Influencer',): 'Specifier / Influencer',
    ('End Customer',): 'End-Customer',
    ('homeowner',): 'Home Owner',
    ('Software/Solution Provider',): 'Software / Solution Provider',
    ('other', 'Other', 'Others'): 'other'
}

for keys, value in inquiry_conditions.items():
    df_train.loc[df_train['inquiry_type'].isin(keys), 'inquiry_type'] = value
    df_test.loc[df_train['inquiry_type'].isin(keys), 'inquiry_type'] = value

for keys, value in customer_type_conditions.items():
    df_train.loc[df_train['customer_type'].isin(keys), 'customer_type'] = value
    df_test.loc[df_train['customer_type'].isin(keys), 'customer_type'] = value

In [74]:
label_columns = [
    #"city",
    #"country",
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

new_df = label_encoding(df_all, label_columns)
df_all

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.00,philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.00,philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.00,india,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.00,india,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.00,india,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64565,0.50,brazil,AS,NaN,40292,NaN,Enterprise,10.0,NaN,NaN,...,LGESP,NaN,0,0,NaN,NaN,NaN,NaN,97,0
64566,0.25,unitedstates,IT,NaN,47466,NaN,Enterprise,0.0,NaN,NaN,...,LGEUS,NaN,0,0,NaN,NaN,NaN,NaN,438,1
64567,0.75,brazil,AS,NaN,46227,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGESP,less than 3 months,0,0,NaN,NaN,NaN,NaN,97,0
64568,0.00,germany,IT,NaN,45667,End Customer,SMB,NaN,NaN,NaN,...,LGEDG,NaN,0,0,NaN,NaN,NaN,NaN,429,0


In [75]:
'''# predict business_area

# label encoding
na_idx = df_all[df_all['business_area'].isna()].index
le = LabelEncoder()
business_area = le.fit_transform(df_all['business_area'])
business_area[na_idx]=-1 # nan=-1
new_df['new_ba'] = business_area

# predict
model_xbgclassifier = XGBClassifier(random_state=seed)

x_ba = new_df[new_df['new_ba']!=-1].drop(columns = ['id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 'ver_cus', 'ver_pro', 'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area', 'new_ba', 'is_converted', 'com_reg_ver_win_rate', 'customer_type'])
y_ba = new_df[new_df['new_ba']!=-1]['new_ba']
model_xbgclassifier.fit(x_ba, y_ba)

value = model_xbgclassifier.predict(new_df.drop(columns = ['id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 'ver_cus', 'ver_pro', 'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area', 'new_ba', 'is_converted', 'com_reg_ver_win_rate', 'customer_type']))
value = le.inverse_transform(value)
df_all['business_area']= value'''

"# predict business_area\n\n# label encoding\nna_idx = df_all[df_all['business_area'].isna()].index\nle = LabelEncoder()\nbusiness_area = le.fit_transform(df_all['business_area'])\nbusiness_area[na_idx]=-1 # nan=-1\nnew_df['new_ba'] = business_area\n\n# predict\nmodel_xbgclassifier = XGBClassifier(random_state=seed)\n\nx_ba = new_df[new_df['new_ba']!=-1].drop(columns = ['id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 'ver_cus', 'ver_pro', 'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area', 'new_ba', 'is_converted', 'com_reg_ver_win_rate', 'customer_type'])\ny_ba = new_df[new_df['new_ba']!=-1]['new_ba']\nmodel_xbgclassifier.fit(x_ba, y_ba)\n\nvalue = model_xbgclassifier.predict(new_df.drop(columns = ['id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 'ver_cus', 'ver_pro', 'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area', 'new_ba', 'is_converted', 'com_reg_ver_win_rate', 'customer_type']))\nvalue = le.inverse_transform(value)\ndf_all['business_are

In [76]:
'''# id_strategic_ver, it_strategic_ver, idit_strategic_ver

df_all.loc[((df_all['business_unit']=='ID') & (df_all['business_area'].isin(['corporate / office', 'hotel & accommodation']))), ['id_strategic_ver']]=1
df_all.loc[~((df_all['business_unit']=='ID') & (df_all['business_area'].isin(['corporate / office', 'hotel & accommodation']))), ['id_strategic_ver']]=0

df_all.loc[(df_all['business_unit']=='IT') & (df_all['business_area'].isin(['corporate / office', 'hotel & accommodation'])), ['it_strategic_ver']]=1
df_all.loc[~((df_all['business_unit']=='IT') & (df_all['business_area'].isin(['corporate / office', 'hotel & accommodation']))), ['it_strategic_ver']]=0

df_all.loc[((df_all['id_strategic_ver']==1) & (df_all['it_strategic_ver']==1)), ['idit_strategic_ver']]=1
df_all.loc[~((df_all['id_strategic_ver']==1) & (df_all['it_strategic_ver']==1)), ['idit_strategic_ver']]=0'''

"# id_strategic_ver, it_strategic_ver, idit_strategic_ver\n\ndf_all.loc[((df_all['business_unit']=='ID') & (df_all['business_area'].isin(['corporate / office', 'hotel & accommodation']))), ['id_strategic_ver']]=1\ndf_all.loc[~((df_all['business_unit']=='ID') & (df_all['business_area'].isin(['corporate / office', 'hotel & accommodation']))), ['id_strategic_ver']]=0\n\ndf_all.loc[(df_all['business_unit']=='IT') & (df_all['business_area'].isin(['corporate / office', 'hotel & accommodation'])), ['it_strategic_ver']]=1\ndf_all.loc[~((df_all['business_unit']=='IT') & (df_all['business_area'].isin(['corporate / office', 'hotel & accommodation']))), ['it_strategic_ver']]=0\n\ndf_all.loc[((df_all['id_strategic_ver']==1) & (df_all['it_strategic_ver']==1)), ['idit_strategic_ver']]=1\ndf_all.loc[~((df_all['id_strategic_ver']==1) & (df_all['it_strategic_ver']==1)), ['idit_strategic_ver']]=0"

In [77]:
# ver_cus

df_all.loc[((df_all['business_area'].isin(['corporate / office', 'hotel & accommodation', 'education', 'retail']))
             &(df_all['customer_type']=='End-Customer')), ['ver_cus']]=1
df_all.loc[~((df_all['business_area'].isin(['corporate / office', 'hotel & accommodation', 'education', 'retail']))
             &(df_all['customer_type']=='End-Customer')), ['ver_cus']]=0

In [78]:
# ver_pro

co = ['standard signage','high brightness signage','interactive signage','video wall signage','led signage','signage care solution','oled signage','special signage','uhd signage','smart tv signage','signage care solutions','digital signage','monitor signage,commercial tv,monior/monitor tv','monitor signage,monior/monitor tv','monitor signage,commercial tv,monior/monitor tv,projector,tv','monitor signage,commercial tv,monior/monitor tv,tv','monitor signage,commercial tv,solar,ess,monior/monitor tv,pc,projector,robot,system ac,ems,rac,chill','tv signage','signage','monitor signage,tv']
ha = ['hotel tv']
rt = ['led signage','video wall signage','high brightness signage','standard signage','oled signage','interactive signage','special signage','smart tv signage','uhd signage','tv signage','signage care solution','ultra stretch signage','monitor signage,monior/monitor tv','monitor signage,commercial tv,solar,ess,monior/monitor tv,pc,projector,robot,system ac,ems,rac,chill','monitor signage,commercial tv,monior/monitor tv,pc,tv,home beauty,audio/video','monitor signage,monior/monitor tv,tv,audio/video','signage','digital signage','signage care solutions','monitor signage,monior/monitor tv,vacuum cleaner,tv,home beauty,commercial tv,pc,refrigerator,styler']

df_all.loc[((df_all['business_area']=='corporate / office')&(df_all['product_category'].isin(co))|(df_all['business_area']=='hotel & accommodation')&(df_all['product_category'].isin(ha))|(df_all['business_area']=='retail')&(df_all['product_category'].isin(rt))), ['ver_pro']]=1
df_all.loc[~((df_all['business_area']=='corporate / office')&(df_all['product_category'].isin(co))|(df_all['business_area']=='hotel & accommodation')&(df_all['product_category'].isin(ha))|(df_all['business_area']=='retail')&(df_all['product_category'].isin(rt))), ['ver_pro']]=0

In [79]:
# ver_win_rate_x
vwrx = {'corporate / office': 0.0030792876608617, 'education': 0.0005719551277132, 'hotel & accommodation': 0.0007167734380046, 'hospital & health care': 6.044033666058328e-05,
        'special purpose': 0.0005432224318428, 'residential (home)': 0.0002983104051378, 'government department': 9.65915660650443e-05, 'retail': 0.0011827288932506,
        'factory': 0.0002153634176709, 'power plant / renewable energy': 2.3159381337232847e-06, 'transportation': 1.2765902883450302e-05, 'public facility': 2.5889552307882245e-05}

for vw in vwrx.keys():
    df_all.loc[(df_all['business_area']==vw), ['ver_win_rate_x']]=vwrx[vw]
    

In [80]:
# ver_win_ratio_per_bu

def new_ratio(df: pd.DataFrame):
    solution = {
        'corporate / office':{'AS': 0.0268456375838926, 'ID': 0.0645661157024793, 'Solution': 0.0344827586206896},
        'education':{'ID': 0.048629531388152, 'AS': 0.0514705882352941},
        'special purpose':{'ID': 0.0640703517587939, 'AS': 0.022633744855967},
        'hospital & health care':{'ID': 0.1311475409836065, 'AS': 0.1285714285714285},
        'residential (home)':{'ID': 0.0354838709677419, 'AS': 0.0201207243460764},
        'government department':{'ID': 0.0794117647058823, 'AS': 0.0227272727272727},
        'retail':{'ID': 0.0498402555910543, 'AS': 0.0115830115830115},
        'hotel & accommodation':{'ID': 0.071345029239766},
        'factory':{'ID': 0.0369127516778523, 'AS': 0.0609243697478991},
        'power plant / renewable energy':{'ID': 0.2857142857142857, 'AS': 0.2272727272727272},
        'transportation':{'ID': 0.0535714285714285},
        'public facility':{'ID': 0.031578947368421, 'AS': 0.0287769784172661}
    }
    
    for business_area in solution.keys():
        for business_unit in solution[business_area].keys():
            df.loc[((df['business_area']==business_area) & (df['business_unit']==business_unit)), ['ver_win_ratio_per_bu']]=solution[business_area][business_unit]
    
    df.loc[(df['ver_win_ratio_per_bu'].isna()), ['ver_win_ratio_per_bu']]=-1

    return df

df_all = new_ratio(df_all)
df_train = df_all[:len(df_train)].reset_index(drop=True)
df_test = df_all[len(df_train):].reset_index(drop=True)

df_train.to_csv('new_train.csv', index=False)
df_test.to_csv('new_test.csv', index=False)

In [81]:
def label_encoding(series: pd.Series) -> pd.Series:
    my_dict = {}
    series = series.astype(str)
    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [82]:
label_columns = [
    "city",
    "country",
    #"customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])
    
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

In [83]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size = 0.2,
    shuffle = True,
    random_state = seed,
)

In [84]:
model_xgb = XGBRegressor(
    n_estimators = iteration,
    eta = 0.01,
    min_child_weight = 50,
    max_depth = 10,
    colsample_bytree = 0.9,
    subsample = 0.9,
    random_state = seed,
    objective = "binary:logistic",
    eval_metric = 'auc'
)

In [85]:
model_xgb.fit(x_train.fillna(-1), y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.9,
             early_stopping_rounds=None, enable_categorical=False, eta=0.01,
             eval_metric='auc', gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.00999999978, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=50,
             missing=nan, monotone_constraints='()', n_estimators=3000,
             n_jobs=0, num_parallel_tree=1, objective='binary:logistic',
             predictor='auto', random_state=42, ...)

In [86]:
pred_xgb = model_xgb.predict(x_val.fillna(-1))
pred_xgb = (pred_xgb > threshold).astype(int)
pred_xgb

array([0, 0, 0, ..., 1, 0, 0])

In [87]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[0, 1])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[0, 1])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[0, 1])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [88]:
get_clf_eval(y_val.astype(int), pred_xgb)

오차행렬:
 [[9928  947]
 [  48  937]]

정확도: 0.9161
정밀도: 0.4973
재현율: 0.9513
F1: 0.6532


In [89]:
x_test = df_test.drop(["is_converted"], axis=1)

In [90]:
test_pred_xgb = model_xgb.predict(x_test.fillna(-1))
test_pred_xgb = (test_pred_xgb > threshold).astype(int)
test_pred_xgb

array([1, 1, 1, ..., 0, 0, 1])

In [91]:
sum(test_pred_xgb) # True로 예측된 개수

2249

In [92]:
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred_xgb

df_sub.to_csv("submission.csv", index=False)
df_sub

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,NaN,...,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,1
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,NaN,NaN,...,LGEUS,NaN,0,0,0.000013,NaN,transportation,Others,437,1
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,NaN,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,1
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,NaN,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,0
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,1.0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,13855,0.50,/São Paulo/Brazil,AS,NaN,40292,NaN,Enterprise,10.0,NaN,...,LGESP,NaN,0,0,NaN,NaN,NaN,NaN,97,0
5267,7979,0.25,General / / United States,IT,NaN,47466,NaN,Enterprise,0.0,NaN,...,LGEUS,NaN,0,0,NaN,NaN,NaN,NaN,438,1
5268,12887,0.75,/ OURO BRANCO / Brazil,AS,NaN,46227,Specifier/ Influencer,Enterprise,NaN,NaN,...,LGESP,less than 3 months,0,0,NaN,NaN,NaN,NaN,97,0
5269,17530,0.00,/ / Germany,IT,NaN,45667,End Customer,SMB,NaN,NaN,...,LGEDG,NaN,0,0,NaN,NaN,NaN,NaN,429,0
